In [2]:
import pandas as pd
import yfinance as yf
import numpy as np
import requests
import io
import time
from scipy.stats import linregress

# Try to import yahooquery for fast filtering
try:
    from yahooquery import Ticker
except ImportError:
    print("Please install yahooquery: pip install yahooquery")

# ==========================================
# 2. STEP 1: FETCH UNIVERSE (Robust Version)
# ==========================================
def get_raw_universe():
    print("--- STEP 1: Fetching Raw Stock Lists ---")
    tickers = []
    
    # 1. USA (NASDAQ Traded List)
    try:
        url = "http://www.nasdaqtrader.com/dynamic/symdir/nasdaqtraded.txt"
        df_us = pd.read_csv(url, sep='|')
        df_us = df_us[(df_us['Test Issue'] == 'N') & (df_us['ETF'] == 'N')]
        us_list = df_us['Symbol'].str.replace('.', '-', regex=False).dropna().unique().tolist()
        us_list = [t for t in us_list if len(t) < 5 and '$' not in t]
        
        tickers.extend(us_list)
        print(f"   -> Found {len(us_list)} US candidates.")
    except Exception as e:
        print(f"   Error fetching US list: {e}")

    # 2. CANADA (TSX Composite - Robust Search)
    try:
        url_ca = "https://en.wikipedia.org/wiki/S%26P/TSX_Composite_Index"
        headers = {"User-Agent": "Mozilla/5.0"} 
        r = requests.get(url_ca, headers=headers)
        
        # FIX: Scan ALL tables to find the one with 'Symbol'
        tables = pd.read_html(io.StringIO(r.text))
        df_ca = pd.DataFrame()
        
        for t in tables:
            if 'Symbol' in t.columns:
                df_ca = t
                break
        
        if not df_ca.empty:
            ca_list = df_ca['Symbol'].apply(lambda x: x.replace('.', '-') + ".TO").tolist()
            tickers.extend(ca_list)
            print(f"   -> Found {len(ca_list)} Canadian candidates.")
        else:
            print("   -> Warning: Could not find Canadian stock table on Wikipedia.")
            
    except Exception as e:
        print(f"   Error fetching Canada list: {e}")

    return list(set(tickers))

# ==========================================
# 3. STEP 2: SPLIT FILTER (With Debug Counts)
# ==========================================
def filter_universe_split(ticker_list):
    print(f"\n--- STEP 2: Applying Advanced Filters (YahooQuery) ---")
    
    # Configuration
    MIN_PRICE = 5.0
    MIN_CAP_US = 300_000_000
    MIN_VOL_US = 1_000_000
    MIN_CAP_CA = 100_000_000
    MIN_VOL_CA = 100_000
    
    MIN_CURRENT_RATIO = 1.0
    MIN_OP_MARGIN = 0.001
    MAX_PE = 100.0
    MAX_BETA = 3.0
    
    valid_candidates = []
    chunk_size = 500 
    
    # Counters for debugging
    cnt_us = 0
    cnt_ca = 0
    
    for i in range(0, len(ticker_list), chunk_size):
        chunk = ticker_list[i:i+chunk_size]
        print(f"   Filtering batch {i} - {min(i+chunk_size, len(ticker_list))}...", end='\r')
        
        try:
            yq = Ticker(chunk, asynchronous=True)
            data = yq.get_modules("summaryDetail defaultKeyStatistics price financialData")
            
            for symbol in chunk:
                if symbol not in data or isinstance(data[symbol], str): continue
                
                # Extract
                price_mod = data[symbol].get('price', {})
                curr_price = price_mod.get('regularMarketPrice', 0) or 0
                mkt_cap = price_mod.get('marketCap', 0) or 0
                
                summ_mod = data[symbol].get('summaryDetail', {})
                avg_vol = summ_mod.get('averageVolume', 0) or 0
                beta = summ_mod.get('beta', 0) or 0
                pe_ratio = summ_mod.get('trailingPE', 0)
                
                fin_mod = data[symbol].get('financialData', {})
                curr_ratio = fin_mod.get('currentRatio', 0) or 0
                op_margin = fin_mod.get('operatingMargins', 0) or 0
                
                # Logic
                if curr_price < MIN_PRICE: continue
                if curr_ratio < MIN_CURRENT_RATIO: continue
                if op_margin < MIN_OP_MARGIN: continue
                
                is_canada = symbol.endswith('.TO')
                
                if is_canada:
                    if mkt_cap < MIN_CAP_CA: continue
                    if avg_vol < MIN_VOL_CA: continue
                else:
                    if mkt_cap < MIN_CAP_US: continue
                    if avg_vol < MIN_VOL_US: continue

                if MAX_BETA and beta > MAX_BETA: continue
                if MAX_PE and pe_ratio and pe_ratio > MAX_PE: continue

                # Add to list
                valid_candidates.append({
                    'Ticker': symbol, 
                    'Price': curr_price,
                    'Market_Cap': mkt_cap,
                    'Region': 'Canada' if is_canada else 'USA',
                    'Current_Ratio': curr_ratio,
                    'Op_Margin': op_margin
                })
                
                # Update Counters
                if is_canada: cnt_ca += 1
                else: cnt_us += 1
                        
        except Exception as e:
            continue
            
    df = pd.DataFrame(valid_candidates)
    print(f"\n   -> Filter complete. Survivors: {len(df)}")
    print(f"      - USA Survivors: {cnt_us}")
    print(f"      - Canada Survivors: {cnt_ca}")
    
    return df

# ==========================================
# 4. STEP 3: DEEP DIVE (Credit Model)
# ==========================================
def run_credit_model(candidates_df):
    if candidates_df.empty: return None, None, None
    
    print(f"\n--- STEP 3: Deep Analysis (Credit Model + Options Check) ---")
    
    fortress, moonshot, distress = [], [], []
    
    for index, row in candidates_df.iterrows():
        ticker = row['Ticker']
        
        if index % 5 == 0:
            print(f"   Analyzing {index}/{len(candidates_df)}...", end='\r')
            time.sleep(0.2)
            
        try:
            stock = yf.Ticker(ticker)
            
            # 1. Option Check (Reliable)
            try:
                if not stock.options: continue 
            except:
                continue 
            
            # 2. Data
            info = stock.info
            fin = stock.financials
            bs = stock.balance_sheet
            
            if fin.empty or bs.empty: continue
            if 'Financial' in info.get('sector', ''): continue
            
            # 3. Metrics
            z = calculate_z_score(info, fin, bs)
            trend = get_margin_trend(fin)
            int_cov = get_interest_coverage(fin)
            roic = calculate_roic(fin, bs)
            
            item = {
                'Ticker': ticker,
                'Price': row['Price'],
                'Region': row['Region'],
                'Z-Score': z,
                'Margin_Trend': trend,
                'Int_Cov': int_cov,
                'ROIC': roic,
                'Current_Ratio': row['Current_Ratio'],
                'Op_Margin': row['Op_Margin']
            }
            
            # 4. Buckets
            if (z > 2.99) and (trend in ["Improving", "Stable"]) and (int_cov > 4.0) and (roic > 5.0):
                fortress.append(item)
            elif (z < 1.8) or (int_cov < 1.5):
                distress.append(item)
            elif (z < 2.5) and (trend == "Improving"):
                moonshot.append(item)
                
        except:
            continue
            
    return pd.DataFrame(fortress), pd.DataFrame(moonshot), pd.DataFrame(distress)

# --- MAIN EXECUTION ---
# 1. Get Universe
raw_tickers = get_raw_universe()

# 2. Split Filter (US vs Canada)
filtered_df = filter_universe_split(raw_tickers)

# 3. Run Analysis
if not filtered_df.empty:
    fortress_df, moonshot_df, distress_df = run_credit_model(filtered_df)
    
    if not fortress_df.empty:
        print("\n\n--- FORTRESS STOCKS (Top Picks) ---")
        display(fortress_df.sort_values(by='Z-Score', ascending=False).head(10))
    
    if not moonshot_df.empty:
        print("\n--- MOONSHOT STOCKS ---")
        display(moonshot_df.sort_values(by='Z-Score', ascending=False).head(10))

--- STEP 1: Fetching Raw Stock Lists ---
   -> Found 5974 US candidates.
   -> Warning: Could not find Canadian stock table on Wikipedia.

--- STEP 2: Applying Advanced Filters (YahooQuery) ---
   Filtering batch 5500 - 5974...
   -> Filter complete. Survivors: 608
      - USA Survivors: 608
      - Canada Survivors: 0

--- STEP 3: Deep Analysis (Credit Model + Options Check) ---
   Analyzing 605/608...

--- FORTRESS STOCKS (Top Picks) ---


,Ticker,Price,Region,Z-Score,Margin_Trend,Int_Cov,ROIC,Current_Ratio,Op_Margin
77,WPM,124.22,USA,206.55,Improving,2569.34,9.30,8.089,0.66542
69,APP,714.23,USA,32.58,Improving,5.89,47.98,3.250,0.76795
44,FAST,41.56,USA,30.85,Stable,206.85,29.96,4.259,0.20696
11,MNST,77.31,USA,29.27,Stable,100.00,30.51,3.185,0.30738
122,OR,37.58,USA,22.54,Improving,16.23,7.37,4.373,0.65183
86,ADMA,19.28,USA,21.37,Improving,9.98,42.37,7.128,0.38005
124,GOOG,314.96,USA,20.79,Improving,419.37,28.71,1.747,0.30512
58,TSEM,121.61,USA,20.13,Stable,45.93,6.88,6.610,0.12783
110,ODFL,159.49,USA,19.18,Stable,7283.01,28.02,1.204,0.25655
14,CPRX,24.35,USA,17.02,Improving,100.00,69.34,6.622,0.44657



--- MOONSHOT STOCKS ---


,Ticker,Price,Region,Z-Score,Margin_Trend,Int_Cov,ROIC,Current_Ratio,Op_Margin
10,EGO,37.18,USA,2.47,Improving,20.69,7.33,2.794,0.39688
16,MBC,11.27,USA,2.47,Improving,3.37,8.28,1.865,0.08499
23,OC,113.74,USA,2.43,Improving,8.02,11.52,1.396,0.18070
5,HAYW,15.94,USA,2.42,Improving,3.16,8.04,3.184,0.16915
4,FLNC,20.21,USA,2.36,Improving,NaN,-28.24,1.506,0.04707
9,FTI,44.65,USA,2.35,Improving,10.09,23.98,1.130,0.15299
1,CXM,7.72,USA,2.34,Improving,100.00,4.65,1.657,0.05515
22,KBR,40.22,USA,2.34,Improving,3.81,10.02,1.177,0.06266
14,VAL,49.51,USA,2.33,Improving,4.28,11.95,1.871,0.21135
15,CCK,102.90,USA,2.32,Improving,3.31,13.64,1.155,0.13991
